In [73]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
import pandas as pd
import numpy as np
from math import sqrt
# Load the music dataset
music_dataset = pd.read_csv('/content/drive/MyDrive/dataset/tcc_ceds_music.csv')

In [75]:

print("\n\n==================Music==========================")
music_dataset = pd.DataFrame(music_dataset)
print(music_dataset)

# Process genres for music
music_dataset['genre'] = music_dataset.genre.str.split('|')
music_types = music_dataset.copy()

# Loop to put 1's in the column that match the genre
for index, row in music_dataset.iterrows():
    for genre in row['genre']:
        music_types.at[index, genre] = 1

# Filling in the NaN values with 0 => Shows the song doesn't have that column's genre
music_types = music_types.fillna(0)
print("==================Top 10 Genres==========================")
print(music_types.head(10))

favt_music = []
while True:
    yes = input("Want to add more favorite songs? (y/n): ")
    if yes.lower() == 'y':
        user_track_name = input("Enter track name: ")
        user_rating = float(input("Enter rating: "))
        dic = {"track_name": user_track_name, "rating": user_rating}
        favt_music.append(dic)
    else:
        break






==================Music==========================
       song_id           artist_name                     track_name  \
0            0                mukesh           mohabbat bhi jhoothi   
1            4         frankie laine                      i believe   
2            6           johnnie ray                            cry   
3           10           pérez prado                       patricia   
4           12  giorgos papadopoulos             apopse eida oneiro   
...        ...                   ...                            ...   
28367    82447               mack 10                10 million ways   
28368    82448                m.o.p.  ante up (robbin hoodz theory)   
28369    82449                  nine                  whutcha want?   
28370    82450            will smith                         switch   
28371    82451                 jeezy                         r.i.p.   

       Unnamed: 3  release_date    genre  \
0             NaN          1950      pop   
1      

In [76]:
print("==================Music From User==========================")
# Creating a DataFrame from user's favorite songs
music_names = pd.DataFrame(favt_music)

# Filtering out the songs by track name
song_matches = music_dataset[music_dataset['track_name'].isin(music_names['track_name'].tolist())]

# Then merging it to get the trackId
music_names = pd.merge(song_matches, music_names, on='track_name')

# Do not need genres from the user input data, so we drop that column
music_names = music_names.drop('genre', axis=1)

# Filtering out the songs from the input
filtering_music = music_types[music_types['track_name'].isin(music_names['track_name'].tolist())]
print(filtering_music[['track_name', 'genre', 'release_date']])

==================Music From User==========================
       track_name      genre  release_date
16     blue skies      [pop]          1951
17      dark eyes      [pop]          1951
530    blue skies      [pop]          1962
5789   blue skies      [pop]          2009
8764   blue skies  [country]          1975
8783   blue skies  [country]          1975
12806  blue skies    [blues]          1959
17081  blue skies    [blues]          2019
17903  blue skies     [jazz]          1970
18889  blue skies     [jazz]          1988
20078  blue skies     [jazz]          2008
20767  blue skies     [jazz]          2017


In [77]:
# Resetting the index to avoid future issues
filtering_music = filtering_music.reset_index(drop=True)

# Dropping unnecessary columns to calculate similarity
genre_list = filtering_music.drop('track_name',1).drop('genre', 1).drop('release_date', 1)

print("==================Music Categories==========================")
print(genre_list)


==================Music Categories==========================
    song_id         artist_name  Unnamed: 3  \
0        40            les paul         0.0   
1        44            les paul         0.0   
2      1388         bobby darin         0.0   
3     16664  noah and the whale         0.0   
4     25279          jim reeves         0.0   
5     25344       willie nelson         0.0   
6     40648     ella fitzgerald         0.0   
7     54280             santana         0.0   
8     56246    django reinhardt         0.0   
9     58628  harry connick, jr.         0.0   
10    61266        susie arioli         0.0   
11    62850         diana krall         0.0   

                                               lyrics  len    dating  \
0   blue sky smile blue sky bluebirds sing song bl...   43  0.001385   
1   light bright dark shit backstage alright cause...  182  0.000358   
2   blue sky smilin nothin blue sky bluebirds sing...   52  0.001350   
3   song break heart song cause blue sk

<ipython-input-77-cc0d06c2cd2b>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  genre_list = filtering_music.drop('track_name',1).drop('genre', 1).drop('release_date', 1)
<ipython-input-77-cc0d06c2cd2b>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  genre_list = filtering_music.drop('track_name',1).drop('genre', 1).drop('release_date', 1)
<ipython-input-77-cc0d06c2cd2b>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  genre_list = filtering_music.drop('track_name',1).drop('genre', 1).drop('release_date', 1)


In [78]:
print("==================User Similar Ratings==========================")
# Exclude non-numeric columns before conversion
numeric_columns = genre_list.select_dtypes(include=[np.number]).columns
genre_list[numeric_columns] = genre_list[numeric_columns].astype(float)

# Calculate user interest
user_interest = genre_list[numeric_columns].transpose().dot(music_names['rating']).reset_index()
user_interest.columns = ['genre', 'rating']

# Filter and display only the desired genres
desired_genres = ['pop', 'country', 'blues', 'jazz', 'reggae', 'rock', 'hip hop']
user_interest = user_interest[user_interest['genre'].isin(desired_genres)]

print(user_interest)

# Now let's get the genres of every song in our original dataframe
music_types_list = music_types.set_index(music_types['song_id'])
# Dropping the unnecessary information
music_types_list = music_types_list.drop('track_name',1).drop('genre', 1).drop('release_date', 1)

# A tuple of array dimensions that tells the number of rows and columns of a given DataFrame
music_types_list.shape


==================User Similar Ratings==========================
      genre  rating
26      pop    16.0
27  country     8.0
28    blues     8.0
29     jazz    17.0
30   reggae     0.0
31     rock     0.0
32  hip hop     0.0


<ipython-input-78-7eb5dc20a9ce>:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  music_types_list = music_types_list.drop('track_name',1).drop('genre', 1).drop('release_date', 1)
<ipython-input-78-7eb5dc20a9ce>:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  music_types_list = music_types_list.drop('track_name',1).drop('genre', 1).drop('release_date', 1)
<ipython-input-78-7eb5dc20a9ce>:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  music_types_list = music_types_list.drop('track_name',1).drop('genre', 1).drop('release_date', 1)


(28372, 36)

In [79]:
# Calculate user profile
print("==================User Profile==========================")
# Get the genres of every track in the original dataframe
music_types_list = music_types.set_index('song_id')
# Drop unnecessary columns
music_types_list = music_types_list.drop('track_name',1).drop('genre', 1).drop('release_date', 1)

# Get the shape of the DataFrame
shape = music_types_list.shape

# print("Shape of music_types_list:", shape)
# Multiply the genres by the weights and then take the weighted average
suggested_tracks = ((music_types_list * user_interest).sum(axis=1)) / (user_interest.sum())

# Sort the recommendations in descending order
suggested_tracks = suggested_tracks.sort_values(ascending=False)


==================User Profile==========================


<ipython-input-79-f54002d46932>:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  music_types_list = music_types_list.drop('track_name',1).drop('genre', 1).drop('release_date', 1)
<ipython-input-79-f54002d46932>:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  music_types_list = music_types_list.drop('track_name',1).drop('genre', 1).drop('release_date', 1)
<ipython-input-79-f54002d46932>:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  music_types_list = music_types_list.drop('track_name',1).drop('genre', 1).drop('release_date', 1)


In [80]:
# Read the ratings dataset
music_ratings = pd.read_csv('/content/drive/MyDrive/dataset/ratings.csv',chunksize=100000)

lst = []
for i in music_ratings:
    lst.append(i)
    break  # Retrieve only the first chunk

music_ratings = pd.concat(lst)
music_ratings = music_ratings.head(1000000)

print("==================Ratings Data Set==========================")
print(music_ratings.head())


==================Ratings Data Set==========================
   user_id  song_id  rating
0        1        0       3
1        1        4       1
2        1        6       2
3        1       10       1
4        1       12       3


In [81]:
dataset = music_dataset.drop('genre', axis=1, errors='ignore')

row_column = music_ratings[music_ratings['song_id'].isin(music_names['song_id'])]

id_user_music_rating = row_column.groupby(['user_id'])
# Sorting it so users with the most music in common with the input will have priority
id_user_music_rating = sorted(id_user_music_rating, key=lambda x: len(x[1]), reverse=True)
id_user_music_rating = id_user_music_rating[0:200]

# Store the Pearson Correlation in a dictionary, where the key is the user ID and the value is the coefficient
# CALCULATING PEARSON CORRELATION COEFFICIENT TO FIND THE SIMILARITY BETWEEN THE TWO USERS
user_similarities_dict = {}

<ipython-input-81-08687dadbde3>:7: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  id_user_music_rating = sorted(id_user_music_rating, key=lambda x: len(x[1]), reverse=True)


In [82]:
# For every user group in our subset
for key, value in id_user_music_rating:
    # Let's start by sorting the input and current user value so the values aren't mixed up later on
    value = value.sort_values(by='song_id')
    music_names = music_names.sort_values(by='song_id')
    # Get the N for the formula
    # nRatings = len(value)
    # Get the review scores for the music that they both have in common
    music_scores = music_names[music_names['song_id'].isin(value['song_id'].tolist())]
    # And then store them in a temporary buffer variable in a list format to facilitate future calculations
    scoring_list = music_scores['rating'].tolist()
    # Let's also put the current user group reviews in a list format
    rating_list = value['rating'].tolist()
    # Now let's calculate the Pearson correlation between two users, so called, x and y
    xi_minus_x = sum([i ** 2 for i in scoring_list]) - pow(sum(scoring_list), 2) / float(len(value))
    yi_minus_y = sum([i ** 2 for i in rating_list]) - pow(sum(rating_list), 2) / float(len(value))
    x_y_product = sum(i * j for i, j in zip(scoring_list, rating_list)) - sum(scoring_list) * sum(rating_list) / float(
        len(value))

    # If the denominator is different than zero, then divide, else, 0 correlation.
    if xi_minus_x != 0 and yi_minus_y != 0:
        user_similarities_dict[key] = x_y_product / sqrt(xi_minus_x * yi_minus_y)
    else:
        user_similarities_dict[key] = 0

user_similarities_dict.items()
Pearson_Correlation = pd.DataFrame.from_dict(user_similarities_dict, orient='index')
Pearson_Correlation.columns = ['similarityIndex']
Pearson_Correlation['user_id'] = Pearson_Correlation.index
Pearson_Correlation.index = range(len(Pearson_Correlation))
print()
print()
print(Pearson_Correlation)




   similarityIndex  user_id
0              0.0     1200
1              0.5        1
2              0.0     1000
3              0.0      880
4              0.0     2034


In [83]:

###########################################################################################################################################

similarityindex_userid = Pearson_Correlation.sort_values(by='similarityIndex', ascending=False)[0:50]

rows_columns = similarityindex_userid.merge(music_ratings, left_on='user_id', right_on='user_id', how='inner')

# Multiplies the similarity by the user's ratings
rows_columns['weightedRating'] = rows_columns['similarityIndex'] * rows_columns['rating']

print("==================User Ratings==========================")
print(rows_columns.head())


==================User Ratings==========================
   similarityIndex  user_id  song_id  rating  weightedRating
0              0.5        1        0       3             1.5
1              0.5        1        4       1             0.5
2              0.5        1        6       2             1.0
3              0.5        1       10       1             0.5
4              0.5        1       12       3             1.5


In [84]:
total_user_ratings_rc = rows_columns.groupby('song_id').sum()[['similarityIndex','weightedRating']]
total_user_ratings_rc.columns = ['sum_Index','sum_rating']
print()
print()
print(total_user_ratings_rc)



         sum_Index  sum_rating
song_id                       
0              0.5         1.5
4              0.5         0.5
6              0.5         1.0
10             0.5         0.5
12             0.5         1.5
...            ...         ...
64729          0.0         0.0
64731          0.0         0.0
64732          0.0         0.0
64734          0.0         0.0
64738          0.0         0.0

[17895 rows x 2 columns]


In [85]:

###########################################################################################################################################
#Creates an empty dataframe
print("====================Weighted Average Recommendation Data Set===================")
recommendation_dataset = pd.DataFrame()
#Now we take the weighted average
recommendation_dataset['Weighted Average Recommendation Score'] = total_user_ratings_rc['sum_rating']/total_user_ratings_rc['sum_Index']
recommendation_dataset['song_id'] = total_user_ratings_rc.index
recommendation_dataset = recommendation_dataset.sort_values(by='Weighted Average Recommendation Score', ascending=False)
print(recommendation_dataset)

====================Weighted Average Recommendation Data Set===================
         Weighted Average Recommendation Score  song_id
song_id                                                
2592                                       5.0     2592
712                                        5.0      712
1688                                       5.0     1688
1410                                       5.0     1410
1064                                       5.0     1064
...                                        ...      ...
64729                                      NaN    64729
64731                                      NaN    64731
64732                                      NaN    64732
64734                                      NaN    64734
64738                                      NaN    64738

[17895 rows x 2 columns]


In [86]:
recommended_music=dataset.loc[dataset['song_id'].isin(recommendation_dataset.head(10)['song_id'].tolist())]
print("====================User Selected songs===================")
print(music_names.head())
print("\n\n==================Recommended songs According to User Input=========================")
print(recommended_music.loc[:,['track_name']].head(11))

====================User Selected songs===================
    song_id         artist_name  track_name  Unnamed: 3  release_date  \
0        40            les paul  blue skies         NaN          1951   
11       44            les paul   dark eyes         NaN          1951   
1      1388         bobby darin  blue skies         NaN          1962   
2     16664  noah and the whale  blue skies         NaN          2009   
3     25279          jim reeves  blue skies         NaN          1975   

                                               lyrics  len    dating  \
0   blue sky smile blue sky bluebirds sing song bl...   43  0.001385   
11  light bright dark shit backstage alright cause...  182  0.000358   
1   blue sky smilin nothin blue sky bluebirds sing...   52  0.001350   
2   song break heart song cause blue sky come know...   43  0.071826   
3   blue sky smilin nothin blue sky bluebirds sing...   53  0.001350   

    violence  world/life  ...  feelings  danceability  loudness  acou